In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import imgaug as ia
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random

def ImageAugmentationByKeras():
    # 定義使用的Augmentation
    img_gen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=45,
        width_shift_range=20,
        height_shift_range=20,
        shear_range=0.8,
        zoom_range=0.5,
        horizontal_flip=True,
        vertical_flip=True,
    )
    width = 224
    height = 224
    batch_size = 4

    img = cv2.imread("Tano.JPG")
    # 將圖片尺寸(h=3339, w=4452) -> (h=224, w=224)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # print(type(img[0, 0, 0]))
    # print(type(img[0][0][0]))
    # print(img.dtype)

    img_origin = img.copy()
    # 將img的資料型態從uint8轉為float32
    img = img.astype(np.float32)

    # 輸入的generator要是四維的，將(224, 224, 3) -> (5, 224, 224, 3)
    img_combine = np.array([img, img, img, img], dtype=np.uint8)
    # print(img_combine.shape)
    batch_gen = img_gen.flow(img_combine, batch_size=4)
    # assert next(batch_gen).shape == (batch_size, width, height, 3)

    plt.figure(figsize=(15, 10))
    i = 1
    for batch in batch_gen:
        # print(batch.shape)
        plt.subplot(1, 5, 1)
        plt.imshow(img_origin) # 原圖
        plt.subplot(1, 5, i+1)
        plt.imshow(batch[0+i-1, :, :, :].astype(np.uint8))
        # plt.imshow(batch[1, :, :, :].astype(np.uint8))
        # plt.imshow(batch[2, :, :, :].astype(np.uint8))
        # plt.imshow(batch[3, :, :, :].astype(np.uint8))
        plt.axis('off')
        i += 1
        if i > 4:
            # or the generator would loop infinitely
            break
    plt.show()

def ImageAugmentationByImgaug():
    img = cv2.imread("Tano.JPG")
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_origin = img.copy()
    print(img[0, 0, 0])
    print(type(img[0, 0, 0]))

    img = img.astype(np.float32)
    print(img[0, 0, 0])
    print(type(img[0, 0, 0]))

    # 創造一個array size==(5, 224, 224, 3)
    images = np.random.randint(0, 255, (5, 224, 224, 3), dtype=np.uint8)
    print(images.shape)
    print(type(images))

    # 水平翻轉機率==1.0
    flipper = iaa.Fliplr(1.0)
    images[0] = flipper.augment_image(img)

    # 垂直翻轉機率40%
    vflipper = iaa.Flipud(p=0.4)
    images[1] = vflipper.augment_image(img)

    # 高斯模糊圖像( sigma of 3.0)
    blurer = iaa.GaussianBlur(3.0)
    images[2] = blurer.augment_image(img)

    # 向左橫移16個像素
    translater = iaa.Affine(translate_px={"x": -16})
    images[3] = translater.augment_image(img)

    # 縮放照片，區間(0.8-1.2倍)
    scaler = iaa.Affine(scale={"y": (0.8, 1.2)})
    images[4] = scaler.augment_image(img)

    i = 1
    plt.figure(figsize=(15, 10))
    for image in images:
        plt.subplot(1, 6, 1)
        plt.imshow(img_origin.astype(np.uint8))
        plt.subplot(1, 6, i + 1)
        plt.imshow(image.astype(np.uint8))
        plt.axis('off')
        i += 1
    plt.show()


def ImageAugmentationByUser():
    img = cv2.imread('Tano.JPG')
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # cv2讀進來是BGR，轉成RGB
    img_origin = img.copy()
    img = np.array(img, dtype=np.float32)

    img_combine = np.array([img, img, img, img], dtype=np.uint8)

    sometimes = lambda aug: iaa.Sometimes(0.5, aug)  # Sometimes(0.5, ...) 代表每次都有50%的機率運用不同的Augmentation

    # 包裝想運用之圖像強化方式
    seq = iaa.Sequential([
        iaa.Crop(px=(0, 16)),
        iaa.Fliplr(0.4),
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-10, 10),
            shear=(-8, 8),
            order=[0, 1],
            cval=(0, 255),
            mode=ia.ALL
        )),
        sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))),
        # convert images into their superpixel representation
        sometimes(iaa.OneOf([
            iaa.GaussianBlur((0, 3.0)),  # blur images
            iaa.AverageBlur(k=(1, 3)),  # blur image using local means with kernel sizes between 1 and 3
            iaa.MedianBlur(k=(3, 5)),  # blur image using local medians with kernel sizes between 3 and 5
        ])),
        sometimes(iaa.Sharpen(alpha=(0, 0.2), lightness=(0.1, 0.4))),  # sharpen images
        sometimes(iaa.Emboss(alpha=(0, 0.3), strength=(0, 0.5))),  # emboss images
    ], random_order=True)

    images_aug = seq.augment_images(img_combine)  ## Image Augmentation
    print(images_aug)
    print(type(images_aug))
    print(images_aug.shape)

    ##畫出來
    i = 1
    plt.figure(figsize=(15, 20))
    for image in images_aug:
        plt.subplot(1, 5, 1)
        plt.imshow(img_origin.astype(np.uint8))
        plt.subplot(1, 5, i + 1)
        plt.imshow(image.astype(np.uint8))
        plt.axis('off')
        i += 1
        if i >= 5:
            break
    plt.show()


"""----------------------------- Self-definition -----------------------------"""
'''隨機改變亮度'''


class RandomBrightness(object):
    '''Function to randomly make image brighter or darker
    Parameters
    ----------
    delta: float
        the bound of random.uniform distribution
    '''

    def __init__(self, delta=16):
        assert 0 <= delta <= 255
        self.delta = delta

    def __call__(self, image):
        delta = random.uniform(-self.delta, self.delta)
        if random.randint(0, 1):
            image = image + delta
        image = np.clip(image, 0.0, 255.0)
        return image


'''隨機改變對比'''
class RandomContrast(object):
    '''Function to strengthen or weaken the contrast in each image
    Parameters
    ----------
    lower: float
        lower bound of random.uniform distribution
    upper: float
        upper bound of random.uniform distribution
    '''
    def __init__(self, lower=0.5, upper=1.5):
        assert upper >= lower, "contrast upper must be >= lower."
        assert lower >= 0, "contrast lower must be non-negative."
        self.lower = lower
        self.upper = upper

    def __call__(self, image):
        alpha = random.uniform(self.lower, self.upper)
        if random.randint(0, 1):
            image = image * alpha
        image = np.clip(image, 0.0, 255.0)
        return image


'''包裝所有Augmentation'''
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image):
        for t in self.transforms:
            image = t(image)
        return image


'''包裝Imgaug'''
class ImgAugSequence(object):
    def __init__(self, sequence):
        self.sequence = sequence

    def __call__(self, image):
        image = self.sequence.augment_image(image)

        return image


class TrainAugmentations(object):
    def __init__(self):
        # Define imgaug.augmenters Sequential transforms
        sometimes = lambda aug: iaa.Sometimes(0.4, aug)  # applies the given augmenter in 50% of all cases

        img_seq = iaa.Sequential([
            sometimes(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.03 * 255), per_channel=0.5)),
            sometimes(iaa.ContrastNormalization((0.5, 2.0), per_channel=1), ),
            sometimes(iaa.Sharpen(alpha=(0, 0.2), lightness=(0.1, 0.4))),  # sharpen images
            sometimes(iaa.Emboss(alpha=(0, 0.3), strength=(0, 0.5))),  # emboss images
        ], random_order=True)

        self.aug_pipeline = Compose([
            RandomBrightness(16),  # make image brighter or darker
            RandomContrast(0.9, 1.1),  # strengthen or weaken the contrast in each image
            ImgAugSequence(img_seq),
        ])

    def __call__(self, image):
        image = self.aug_pipeline(image)
        return image
"""----------------------------- Self-definition -----------------------------"""


if __name__ == "__main__":
    # ImageAugmentationByKeras()
    # ImageAugmentationByImgaug()
    # ImageAugmentationByUser()

